In [ ]:
!pip install keras-tuner

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, SimpleRNN, Conv1D, GlobalMaxPooling1D, Flatten, MaxPooling1D
from tensorflow.keras.layers import Bidirectional

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.optimizers import SGD, Adam

import time
import keras_tuner as kt

In [ ]:
# load the dataset from static csv
# df = pd.read_csv('data/PFG.csv', header=0, index_col=0, parse_dates=True, usecols=['Date','Adj Close'])
# df.head(2)

### Load Data using Pandas Datareader

In [ ]:
!pip install pandas-datareader

In [ ]:
!pip install yfinance --upgrade --no-cache-dir

In [ ]:
import pandas_datareader.data as pdr
import yfinance as yf

yf.pdr_override()
df = pdr.get_data_yahoo('PFG')#, start, end)

df = df[['Adj Close']]  
df.head()

In [ ]:
df.tail()

In [ ]:
dataset = df.values.astype('float32')
dataset

In [ ]:
dataset.shape

In [ ]:
split_pct = 0.8

split = int(split_pct*len(dataset))

train = dataset[:split]
valid = dataset[split:]

print(len(train))
print(len(valid))

train.shape, valid.shape

### Hyperparameter Tuning for Multiple Lags

In [ ]:
def hpo_process(train, valid, lag_list, scaler=None, random_tuner=True, epochs=20, SEED=1, MAX_TRIALS=10, EXECUTION_PER_TRIAL=2, BAYESIAN_NUM_INITIAL_POINTS=1, train_batch=1, valid_batch=1):
    # directory for each search
    RANDOM_DIR = "random-search-{}".format(int(time.time()))
    BAYESIAN_DIR = "bayesian-search-{}".format(int(time.time()))
    
    results={}
    
    for lag in lag_list:
        def trial_model_builder(hp):
            model = Sequential()
            model.add(LSTM(units=hp.Choice('lstm1_units', values=[64,128,256]),
                           dropout=hp.Float('lstm1_do_rate', min_value=0, max_value=.3, sampling='linear'),
                           input_shape=(lag, 1)))
            model.add(Dense(1))
            model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])))
            return model
        
        print('Tuning for {} lags'.format(lag))

        if scaler is not None:
            train = scaler.fit_transform(train)
            valid = scaler.transform(valid)
        
        trial_name = "ts_tuner_lags_{}".format(lag)
        train_generator = TimeseriesGenerator(train, train, length=lag, batch_size=train_batch)
        valid_generator = TimeseriesGenerator(valid, valid, length=lag, batch_size=valid_batch)

        if random_tuner:
            tuner = kt.RandomSearch(trial_model_builder, objective='val_loss', seed=SEED, max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL
                                    ,directory=RANDOM_DIR, project_name=trial_name)
        else:
            tuner = kt.BayesianOptimization(trial_model_builder, objective='val_loss', num_initial_points=BAYESIAN_NUM_INITIAL_POINTS, seed=SEED
                                            ,max_trials=MAX_TRIALS, executions_per_trial=EXECUTION_PER_TRIAL, directory=BAYESIAN_DIR, project_name=trial_name)
            
        tuner.search(train_generator, epochs=epochs, verbose=1, validation_data=valid_generator)
        
        results[str(lag)]=tuner.get_best_hyperparameters(num_trials=1)[0]
    return results

In [ ]:
results = hpo_process(train, valid, [3,5], scaler=MinMaxScaler(), epochs=5, MAX_TRIALS=2, train_batch=5)

In [ ]:
results['3'].get('lstm1_units')

In [ ]:
results['3'].get('lstm1_do_rate')